# Database Project (SWE3033) (Fall 2023)
# HW3 (100pts, Due date: 11/3)

**Instruction:** In this homework, we provide you with a jupyter notebook file (DBP_HW3.ipynb). You should follow the instructions in these documents carefully.

**Submit two files as follows**:
- `DBP_HW3_StudentID.zip`
	- `DBP_HW3_StudentID.ipynb`
	- `DBP_HW3_StudentID.pdf`
   

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=97565a24290e9a1853b45acc42c9c2e6bc109931b00621ac36548a16123c85f0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Import required library

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql import Row

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# 1.

#### (a) Create a DataFrame with the given data and display the generated DataFrame.

In [3]:
# ============= EDIT HERE =============

df = sc.parallelize([
    Row(phase='packing', model='book_pro', serial='book_pro1'),
    Row(phase='packing', model='book_pro', serial='book_pro2'),
    Row(phase='packing', model='plus', serial='plus1'),
    Row(phase='packing', model='book_pro', serial='book_pro3'),
    Row(phase='packing', model='plus', serial='plus2'),
    Row(phase='inspection', model='book_pro', serial='book_pro4'),
    Row(phase='inspection', model='plus', serial='plus3'),
    Row(phase='inspection', model='book_pro', serial='book_pro5'),
    Row(phase='inspection', model='book_pro', serial='book_pro6')
]).toDF()

# =====================================

df.show(truncate=False)

+----------+--------+---------+
|phase     |model   |serial   |
+----------+--------+---------+
|packing   |book_pro|book_pro1|
|packing   |book_pro|book_pro2|
|packing   |plus    |plus1    |
|packing   |book_pro|book_pro3|
|packing   |plus    |plus2    |
|inspection|book_pro|book_pro4|
|inspection|plus    |plus3    |
|inspection|book_pro|book_pro5|
|inspection|book_pro|book_pro6|
+----------+--------+---------+



#### (b) After adding two laptops to the manufacturing process, find the number of products for each model.


In [4]:
# ============= EDIT HERE =============

df2 = sc.parallelize([
    Row(phase='assembly', model='book_pro', serial='book_pro7'),
    Row(phase='assembly', model='plus', serial='plus4')
]).toDF()

new_df = df.unionByName(df2).groupBy("model").count()

# =====================================
new_df.show(truncate=False)

+--------+-----+
|model   |count|
+--------+-----+
|book_pro|7    |
|plus    |4    |
+--------+-----+



#### (c) Group the data in the joined DataFrame by ‘phases’ and count the number of data for each phase.

In [5]:
# ============= EDIT HERE =============
group_df = df.unionByName(df2).groupBy("phase").count()

# =====================================
group_df.show(truncate=False)

+----------+-----+
|phase     |count|
+----------+-----+
|packing   |5    |
|inspection|4    |
|assembly  |2    |
+----------+-----+



## 2.


#### (a) Create a DataFrame for the two given data and join Data 1 with Data 2 using an inner join based on the 'serial' column. (left side: Data 2, right side: Data 1)

In [6]:
# ============= EDIT HERE =============

df = sc.parallelize([
    Row(phase='packing', model='book_pro', serial='book_pro1'),
    Row(phase='packing', model='book_pro', serial='book_pro2'),
    Row(phase='packing', model='plus', serial='plus1'),
    Row(phase='packing', model='book_pro', serial='book_pro3'),
    Row(phase='packing', model='plus', serial='plus2'),
    Row(phase='inspection', model='book_pro', serial='book_pro4'),
    Row(phase='inspection', model='plus', serial='plus3'),
    Row(phase='inspection', model='book_pro', serial='book_pro5'),
    Row(phase='inspection', model='book_pro', serial='book_pro6')
]).toDF()

df2 = sc.parallelize([
    Row(Customer='Lion', serial='book_pro1', due_date='2023-10-25'),
    Row(Customer='Rabbit', serial='book_pro2', due_date='2023-10-27'),
    Row(Customer='Rathor panda', serial='plus1', due_date='2023-10-22'),
    Row(Customer='Fubao', serial='book_pro3', due_date='2023-10-28'),
    Row(Customer='Chinchilla', serial='plus2', due_date='2023-10-22'),
    Row(Customer='Sloth', serial='book_pro4', due_date='2023-10-26')
]).toDF()

joined_df = df2.join(df, df2["serial"] == df["serial"])

# =====================================

joined_df.show(truncate=False)

+------------+---------+----------+----------+--------+---------+
|Customer    |serial   |due_date  |phase     |model   |serial   |
+------------+---------+----------+----------+--------+---------+
|Lion        |book_pro1|2023-10-25|packing   |book_pro|book_pro1|
|Rabbit      |book_pro2|2023-10-27|packing   |book_pro|book_pro2|
|Fubao       |book_pro3|2023-10-28|packing   |book_pro|book_pro3|
|Sloth       |book_pro4|2023-10-26|inspection|book_pro|book_pro4|
|Rathor panda|plus1    |2023-10-22|packing   |plus    |plus1    |
|Chinchilla  |plus2    |2023-10-22|packing   |plus    |plus2    |
+------------+---------+----------+----------+--------+---------+



#### (b) Use an SQL query to select the data from the joined DataFrame where the ‘due_date’ is on or after ‘2023-10-25’. And briefly explain the method you used.

In [7]:
# ============= EDIT HERE =============
joined_df.createOrReplaceTempView("joined_df")

sql_query = """
    SELECT *
    FROM joined_df
    WHERE due_date >= '2023-10-25'
"""

sql_df = spark.sql(sql_query)

# =====================================

sql_df.show(truncate=False)

+--------+---------+----------+----------+--------+---------+
|Customer|serial   |due_date  |phase     |model   |serial   |
+--------+---------+----------+----------+--------+---------+
|Lion    |book_pro1|2023-10-25|packing   |book_pro|book_pro1|
|Rabbit  |book_pro2|2023-10-27|packing   |book_pro|book_pro2|
|Fubao   |book_pro3|2023-10-28|packing   |book_pro|book_pro3|
|Sloth   |book_pro4|2023-10-26|inspection|book_pro|book_pro4|
+--------+---------+----------+----------+--------+---------+



## 3.

In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.appName("ClassificationPractice").getOrCreate()

###  (a) Load the provided dataset, convert it into a DataFrame, and show it.

In [9]:
def create_dataframe(data_path):

    fasion_mnist_data = spark.read.option('header', 'true').option('inferSchema', 'true').csv(data_path)

    col_lst = ['pixel'+str(i+1) for i in range(784)]

    # ============= EDIT HERE =============

    # Instruction 1: Assemble the features into a vector column and name the column "features"
    assembler = VectorAssembler(inputCols=col_lst, outputCol="features")
    fasion_mnist_data = assembler.transform(fasion_mnist_data)

    # Instruction 2: Rename the target column to "label"
    fasion_mnist_data = fasion_mnist_data.withColumnRenamed("label", "label")

    # =====================================

    return fasion_mnist_data

In [10]:
# ============= EDIT HERE =============
# OPTION: You can also change the data path.
train_data_path = 'fashion-mnist_train.csv'
test_data_path = 'fashion-mnist_test.csv'
# =====================================
train_data = create_dataframe(train_data_path)
test_data = create_dataframe(test_data_path)

train_data.show()
test_data.show()

+-----+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---

In [11]:
# Multi-class classification evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

### Logistic Regression
Reference: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html

In [18]:
from pyspark.ml.classification import LogisticRegression

# Training and Test
# ============= EDIT HERE =============
lr = LogisticRegression(maxIter=100, regParam=0.05, elasticNetParam=0.5)
lr_model = lr.fit(train_data)
lr_preds = lr_model.transform(test_data)
lr_accuracy = evaluator.evaluate(lr_preds)

# =====================================

print(f"Accuracy: {lr_accuracy}")

Accuracy: 0.7560803665844201


### Decision Tree
Reference: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html

In [20]:
from pyspark.ml.classification import DecisionTreeClassifier

# On model declaration, fix seed, maxDepth,  to the following values
seed = 2023

# ============= EDIT HERE =============

dt = DecisionTreeClassifier(seed=seed, maxDepth=15)
dt_model = dt.fit(train_data)
dt_preds = dt_model.transform(test_data)
dt_accuracy = evaluator.evaluate(dt_preds)

# =====================================

print(f"Accuracy: {dt_accuracy}")

Accuracy: 0.7440958759252732


### Random Forest
Reference: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html

In [12]:
from pyspark.ml.classification import RandomForestClassifier

# On model declaration, fix seed to the following values
seed = 2023

# ============= EDIT HERE =============
rfc = RandomForestClassifier(seed=seed)
rfc_model = rfc.fit(train_data)
rfc_preds = rfc_model.transform(test_data)
rfc_accuracy = evaluator.evaluate(rfc_preds)

# =====================================

print(f"Accuracy: {rfc_accuracy}")

Accuracy: 0.7724121724826695


# Make sure to save both the code and output values prior to submission.